## Convolutional DNN

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from attack_utils import get_synonymes#, attack, predict_sentence
import random
import nltk
import operator
from tqdm import tqdm
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input,LSTM,Embedding,Flatten,Dense
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation,TimeDistributed
from keras.layers import Bidirectional, LSTM
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate,BatchNormalization,MaxPooling1D, Convolution1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, RepeatVector, Permute, merge
from keras import backend as K
from keras import layers
from keras.models import Sequential, Model

[nltk_data] Downloading package punkt to /home/rushabh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/rushabh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/rushabh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [2]:
MAX_SEQUENCE_LENGTH = 5000
MAX_NUM_WORDS = 25000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2
SAVE_MODEL_PATH = './save/model/'
TEXT_DATA = 'data/fake_or_real_news.csv'
EMBEDDING_FILE=f'glove.6B.{EMBEDDING_DIM}d.txt'

In [3]:
# read in our data and preprocess it

df = pd.read_csv(TEXT_DATA)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with lengths gt 0 -- there are some texts with len = 0
mask = list(df['text'].apply(lambda x: len(x) > 0))
df = df[mask]

In [4]:
# prepare text samples and their labels

texts = df['text']
labels = df['label']

print('Found %s texts.' %texts.shape[0])

Found 6335 texts.


In [5]:
# vectorize the text samples into a 2D integer tensor 

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')

print('Found %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 98817 unique tokens.
Shape of data tensor: (6335, 5000)
Shape of label tensor: (6335,)


In [6]:
# split the data into a training set and a validation set   

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data, 
                                                  labels.apply(lambda x: 0 if x == 'FAKE' else 1), 
                                                  test_size=TEST_SPLIT)

In [8]:
# Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

In [9]:

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

word_index = tokenizer.word_index
nb_words = min(MAX_NUM_WORDS, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/media/rushabh/B4A46314A462D87E/aml/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
BATCH_SIZE = 128

# convs = []
# filter_sizes = [3,4,5]
model=None
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


embedded_sequences = embedding_layer(sequence_input)
# l_lstm = Bidirectional(LSTM(100, return_sequences=True))(embedded_sequences)
l_lstm = LSTM(100, return_sequences=True)(embedded_sequences)
f = Flatten()(l_lstm)
# preds = Dense(1, activation='softmax')(f)
preds = Dense(1, activation='sigmoid')(f)
# preds = TimeDistributed(2, Dense(200,activation='relu'))(l_gru)
model = Model(sequence_input, preds)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - LSTM network")
# checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)
model.summary()

model fitting - LSTM network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5000, 300)         7500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 5000, 100)         160400    
_________________________________________________________________
flatten_1 (Flatten)          (None, 500000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 500001    
Total params: 8,160,401
Trainable params: 660,401
Non-trainable params: 7,500,000
_________________________________________________________________


In [11]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)
    
    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)
    
    return train_acc, test_acc

In [12]:
# define a function that allows us to evaluate our models

from sklearn.metrics import accuracy_score

def evaluate_ourfakes(model, Fake_texts):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    missclasified_fakes = []
    correct_clasified_fakes = []
    def predict(X):
        return np.rint(model.predict(X))
    # training error
    y_predict = predict(Fake_texts)
    y = np.zeros(len(y_predict))
    for i in range(len(y_predict)):
        missclasified_fakes.append(Fake_texts[i]) if y_predict[i] == 1 else correct_clasified_fakes.append(Fake_texts[i])
    acc = accuracy_score(y,y_predict) #model.evaluate(Fake_texts,y)
    return acc, missclasified_fakes, correct_clasified_fakes

In [13]:
model.load_weights(SAVE_MODEL_PATH+'glove_LSTM_model.h5') # replace X with the last saved checkpoint number


In [14]:
def predict_sentence(model,sent):
    try_vector = tokenizer.texts_to_sequences([sent])
    try_vector = pad_sequences([try_vector[0]], 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')
    val = model.predict(try_vector)
    return val


In [15]:
faker = list(df['label'].apply(lambda x: x == 'FAKE'))
fakes = df[faker]
fake_texts = fakes['text']
# fake_labels = fakes['label']

In [16]:
fake_text_vectors = tokenizer.texts_to_sequences(fake_texts)
fake_text_vectors = pad_sequences(fake_text_vectors, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')

In [17]:
acc, missclasified_fakes, correct_clasified_fakes = evaluate_ourfakes(model, fake_text_vectors)

In [18]:
print("total missclassified fakes",len(missclasified_fakes),"total correctly classified fakes",len(correct_clasified_fakes))

total missclassified fakes 120 total correctly classified fakes 3044


In [19]:
correct_fakes =tokenizer.sequences_to_texts(correct_clasified_fakes)

In [20]:
# with open(f'CNN1d_correct_fakes.txt','w', encoding="utf-8") as f:
#     f.writelines(correct_fakes)

In [21]:
# with open(f'CNN1d_correct_fakes.txt','r', encoding="utf-8") as f:
#     correct_fakes = f.readlines()

In [26]:
def attack(dummy,pertub=1, printSwaps = False):    
    dummy_temp = nltk.word_tokenize(dummy[:MAX_SEQUENCE_LENGTH])
    done = False
    unsuccessfullSwaps = 0
    for i in range(int(len(dummy_temp)*pertub)):
        if done:
            break
        itter = 0
        while itter<10000:
            itter+=1
            v = random.randint(0,len(dummy_temp)-1)
            syns,_ = get_synonymes(dummy_temp[v])
            if len(syns)>0:
                break
        if itter == 10000:
            print("didnt catch any synonymes")
            continue
        candidates = {}
        # candidate is a scored candidate dictionary storing all the 
        # synonymes with prediction Score
        word = dummy_temp[v]
        sent = " ".join(dummy_temp)
        candidates[word]=predict_sentence(model,sent)
        for s in syns:
            dummy_temp[v] = s
            sent = (" ".join(dummy_temp))
            val = predict_sentence(model,sent)
            candidates[s] = val
            if val>=0.5:
#                 print("got changes in ",i,val)
                done = True
                break
    #         best_candidate = max(candidates.iteritems(), key=operator.itemgetter(1))[0]
        best_candidate = max(candidates, key=candidates.get)
        if word == best_candidate:
            unsuccessfullSwaps+=1
#             if printSwaps:
#                 print("did not swap")
#         else:
#             if printSwaps:
#             print(word,best_candidate)
        dummy_temp[v] = best_candidate
    if not done:
#         print("sorry")
        return 0,None
    print("swaps done ",i+1-unsuccessfullSwaps,"total words",len(dummy_temp),"unsucessfull swaps try",unsuccessfullSwaps)
    return i+1-unsuccessfullSwaps," ".join(dummy_temp)

In [27]:
success_on = np.zeros(len(correct_fakes),dtype='int16')
# for i in tqdm(range(len(correct_fakes))):
# for i in tqdm(range(300)):
for i in tqdm(range(200)):
    success_on[i],_ = attack(correct_fakes[i],1)



  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 1/200 [02:29<8:14:32, 149.11s/it]

swaps done  27 total words 926 unsucessfull swaps try 4




  1%|          | 2/200 [10:33<13:43:29, 249.54s/it]

swaps done  57 total words 455 unsucessfull swaps try 31




  2%|▏         | 3/200 [10:34<9:34:40, 175.03s/it] 

...

swaps done  59 total words 895 unsucessfull swaps try 21




100%|█████████▉| 199/200 [19:19:14<06:34, 394.95s/it]

swaps done  100 total words 376 unsucessfull swaps try 46




100%|██████████| 200/200 [19:27:35<00:00, 350.28s/it]

swaps done  66 total words 867 unsucessfull swaps try 24


In [28]:
success_on.dump(SAVE_MODEL_PATH+'glove_LSTM_success_rate.npy')

In [29]:
sum(success_on[:200]>0)

199

In [30]:
attack(correct_fakes[0],1)

swaps done  17 total words 926 unsucessfull swaps try 9


(17,
 'daniel greenfield a journalism fellow at the freedom center is a new york writer focusing on radical islam in the final stretch of the election hillary rodham clinton has gone to war with the fbi the word “ unprecedented ” has been thrown around so often this election that it ought to constitute retired but it ’ s still unprecedented for the nominee of a major political party to go war with the fbi but that ’ s exactly what hillary and her people have done patients just waking up now and watching an hour of cnn from their hospital beds would assume that fbi director james comey is hillary ’ s opponent in this election the fbi is under attack by everyone from obama to cnn hillary ’ s people have circulated a letter attacking comey there are currently more media hit pieces lambasting him than targeting trump it wouldn ’ t be too surprising if the clintons or their allies embody to start running attack ads against the fbi the fbi ’ mho leadership is being warned that the entire lef

In [48]:
success_log = np.zeros((len(correct_fakes),3))

In [49]:
success_log.shape

(3044, 3)

In [71]:
import nltk
for i in range(len(correct_fakes)):
    success_log[i][0] = len(nltk.word_tokenize(correct_fakes[i][:MAX_SEQUENCE_LENGTH]))

In [69]:
success_log.T[1] = success_on

In [72]:
success_log.T[0] 

array([926., 455., 427., ..., 438., 867., 862.])

In [85]:
success_log.T[2] = success_log.T[1]/success_log.T[0]*100

/media/rushabh/B4A46314A462D87E/aml/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [87]:
success_on.dump(SAVE_MODEL_PATH+'glove_LSTM_success_rate.npy')